In [1]:
import numpy as np
import os

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms


from torch.autograd import Variable

from features.extractor import get_features
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'conv',
 'conv features extract + lstm.ipynb',
 'data',
 'dataset_UCF-101.ipynb',
 'features',
 'Obj_detector',
 'other_tutorials',
 'README.md',
 'requirements.txt',
 'RNN + loader + basic training.ipynb',
 'RNN Tutorial 1.ipynb',
 'rnn.pkl',
 'transfer_learning_tutorial.ipynb']

In [11]:
# Hyper Parameters
hidden_size = 128
input_size = 512
num_layers = 2
batch_size = 2
num_epochs = 2
learning_rate = 0.01
num_classes = 3
label_str_to_int = {'ApplyEyeMakeup': 0, 'Archery': 1, 'ApplyLipstick': 2}

In [3]:
# def check_size(address, sequence_size=50):
#     x = np.load(address)
#     print (x.shape)
#     if x.shape[0] < sequence_size:
#         return np.concatenate((x, np.zeros((sequence_size - x.shape[0], x.shape[1]))), axis=0)
#     return x

# # Data Loader (Input Pipeline)
# def get_train_loader(path='data/features'):
#     """
#     Function reads .npy files from path.
#     Returns:
#         dataloader, data classes (list), size of input object [n_sequence, n_features], lenght_of_dataset
#     """
#     inputs = []
#     targets = []
#     data_classes = os.listdir(path)
#     label_int = 0
#     for folder in data_classes:
#         current_dir = path + '/' + folder + '/'
        
#         temp = [
#             torch.Tensor(check_size(current_dir +  f, sequence_size=51)) for f in os.listdir(current_dir)
#         ]  # Transform to torch tensors
        
#         targets += ([torch.LongTensor([label_int])] * len(temp))
#         inputs += temp
        
#         label_int += 1
        
#     tensor_x = torch.stack(inputs)
#     tensor_y = torch.stack(targets)
#     print (tensor_x.size())
#     my_dataset = torch.utils.data.TensorDataset(tensor_x, tensor_y)  # Create your datset
#     my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size)  # Create your dataloader
    
    
#     return my_dataloader, data_classes, inputs[-1].size(), len(inputs)

# train_loader, data_classes, [sequence_length, input_size], lenght_of_dataset = get_train_loader()

# num_classes = len(data_classes)
# print(train_loader, data_classes, sequence_length, input_size)

In [29]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        # Forward propagate RNN
        print (x)
        print (h0, c0)
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

# class FinalModel_1(nn.Module):
#     """ Conv net + LSTM """
#     def __init__(self, pretrained_model, input_size, hidden_size, num_layers, num_classes):
#         self.pretrained_model = pretrained_model
#         self.last_layer = RNN(input_size, hidden_size, num_layers, num_classes)

#     def forward(self, x):
#         return self.last_layer(self.pretrained_model(x))
rnn = RNN(input_size=512, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)
rnn
# pretrained_model = torchvision.models.resnet18(pretrained=True)
# model = FinalModel_1(pretrained_model=torch.resnet18(), input_size, hidden_size, num_layers, num_classes)

RNN (
  (lstm): LSTM(512, 128, num_layers=2, batch_first=True)
  (fc): Linear (128 -> 3)
)

In [30]:
# for a, b in  get_features(in_dir='data/data_subset/', batch_size=2):
#     print(a, b.size())
#     break

Loading movie with category Archery and 343 frames
Loading movie with category Archery and 292 frames


['Archery', 'Archery'] torch.Size([2, 35, 512])


In [31]:
label_str_to_int['ApplyEyeMakeup']

0

In [32]:
sequence_length = 35

In [33]:
print ("GPU is available: ", torch.cuda.is_available())

GPU is available:  True


In [34]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [36]:
for epoch in range(num_epochs):
    for i, (labels, images) in enumerate(get_features(in_dir='data/data_subset/', batch_size=1)):
        print (images.size())
        labels = torch.LongTensor([label_str_to_int[i] for i in labels]).cuda()
        
        print (type(images), type(labels))
#         print (images)
#         print(labels)

        images = Variable(images.view(-1, sequence_length, input_size).cuda())
        labels = Variable(labels.view(-1).cuda())

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, lenght_of_dataset//batch_size, loss.data[0]))


Loading movie with category Archery and 67 frames


torch.Size([1, 35, 512])
<class 'torch.cuda.FloatTensor'> <class 'torch.cuda.LongTensor'>
Variable containing:
( 0 ,.,.) = 
  0.6243  0.7202  0.5131  ...   0.2361  0.2989  0.0142
  0.5349  0.4104  0.5383  ...   0.3549  0.2454  0.0000
  0.1639  0.1940  0.3822  ...   0.1800  0.3370  0.0116
           ...             ⋱             ...          
  0.4448  1.1028  0.6127  ...   0.2874  0.1375  0.0014
  0.2120  1.0051  0.8037  ...   0.4565  0.0466  0.0000
  0.4653  0.9591  0.6206  ...   0.6195  0.0439  0.0000
[torch.cuda.FloatTensor of size 1x35x512 (GPU 0)]

Variable containing:
( 0 ,.,.) = 

Columns 0 to 18 
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

Columns 19 to 37 
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

Columns 38 to 56 
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

Columns 57 to 75 
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

Columns 76 to 94 


AssertionError: 

In [7]:
# # Test set is not implemented yet

# # Test the Model
# correct = 0
# total = 0
# for images, labels in test_loader:
#     images = Variable(images.view(-1, sequence_length, input_size))
#     outputs = rnn(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum()

# print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# # Save the Model
# # torch.save(rnn.state_dict(), 'rnn.pkl')
# # torch.save(rnn, 'rnn.pt')

# Generate dummy data set

In [13]:
import numpy as np

In [14]:
n_feat = 100
n_seq = 50 # fixed for now
n_video = 10

In [15]:
for i in range(4,5):
    tmp_data = np.random.random((n_seq+1, n_feat))
    np.save('data/features/Archery/v' + str(i) + '.npy', tmp_data)